# Covalent QuHack Workshop


The goal of this workshop is to showcase how one can install & set up covalent as well as how to use the plugin ecosystem to interact with cloud compute resources. We will run through several examples of workflows using covalent locally, a remote deployment of covalent on AWS, and workflows that utilize AWS plugins. Lastly we will cover an example using Qiskit & Pennylane with Covalent to evaluate Quantum Circuits.

The structure is as follows:
1. How to use covalent locally 
2. How to use AWS executor plugins
3. Host covalent on AWS
4. Use executor plugins on hosted covalent
5. Example workflow using Qiskit & Pennylane


Some useful links include:
- https://github.com/AgnostiqHQ/covalent Covalent Github Repo
- https://covalent.readthedocs.io/en/latest/getting_started/quick_start/index.html Getting Started Guide

## Getting Started with Covalent

To install covalent install it with pip:

In [ ]:
!pip install covalent --no-cache

To start covalent run the following in your terminal:

**$ covalent start** 

> Note: This may not work if run in the notebook so be sure to use your shell in the same virtualenv as your notebook.


In [ ]:
# Say we have a simple script as follows
def add(x, y):
   return x + y

def multiply(x, y):
   return x*y

def divide(x, y):
   return x/y

def workflow(x, y):
   r1 = add(x, y)
   r2 = [multiply(r1, y) for _ in range(4)]
   r3 = [divide(x, value) for value in r2]
   return r1,r2,r3

print(workflow(1,2))

In [ ]:
# With no code changes we add decorators to covert to a covalent workflow

import covalent as ct

@ct.electron
def add(x, y):
   return x + y

@ct.electron
def multiply(x, y):
   return x*y

@ct.electron
def divide(x, y):
   return x/y

@ct.lattice
def workflow(x, y):
   r1 = add(x, y)
   r2 = [multiply(r1, y) for _ in range(4)]
   r3 = [divide(x, value) for value in r2]
   return r1,r2,r3


In [ ]:
dispatch_id = ct.dispatch(workflow)(1, 2)
result = ct.get_result(dispatch_id, wait=True)
print(result.result)

##  AWS Account Setup 

In order to interact with AWS services from your machine you must:

1. Have an AWS Account (Free): https://aws.amazon.com/free/
2. Install the AWS CLI: https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html

Furthermore, if you don't already have one you will need to create an account. Once you are logged in you will want to go to IAM and then to Manage Security Credentials to obtain an `aws_access_key_id` and `aws_secret_access_key`. At which point you will need to store your credentials locally by using the cli:

```bash
$ aws configure
```

Alternatively you can go to `~/.aws/credentials` and create a section:

```
[default]
aws_access_key_id=<ACCESS_KEY>
aws_secret_access_key=<SECRET_ACCESS_KEY>
```

## Using AWS Executor Plugins in Covalent (AWS Batch)

It is recommended to read our documentation of [AWS Executor Plugins](https://covalent.readthedocs.io/en/latest/api/executors/awsplugins.html)

### We will now show a toy example of training an SVM ML model using a covalent workflow (without executors)


In [ ]:
!pip install numpy==1.23.2 scikit-learn==1.1.2

In [ ]:
# Train SVM w/out Executor

from numpy.random import permutation
from sklearn import svm, datasets
import covalent as ct

@ct.electron
def train_svm(data, C, gamma):
    X, y = data
    clf = svm.SVC(C=C, gamma=gamma)
    clf.fit(X[90:], y[90:])
    return clf

@ct.electron
def load_data():
    iris = datasets.load_iris()
    perm = permutation(iris.target.size)
    iris.data = iris.data[perm]
    iris.target = iris.target[perm]
    return iris.data, iris.target

@ct.electron
def score_svm(data, clf):
    X_test, y_test = data
    return clf.score(
      X_test[:90],
    y_test[:90]
    )

@ct.lattice
def run_experiment(C=1.0, gamma=0.7):
    data = load_data()
    clf = train_svm(
      data=data,
      C=C,
      gamma=gamma
    )
    score = score_svm(
      data=data,
    clf=clf
    )
    return score

# Dispatch the workflow
dispatch_id = ct.dispatch(run_experiment)(
  C=1.0,
  gamma=0.7
)

# Wait for our result and get result value
result = ct.get_result(dispatch_id=dispatch_id, wait=True).result

# Get Mean Accuracy of our model
print(result)

### We will now use the AWS Batch executor plugin


In [ ]:
!pip install covalent-awsbatch-plugin --no-cache # Note: we need to restart covalent after installing plugins

In [ ]:
# Import Executor (AWS Batch)

import covalent as ct
from covalent.executor import AWSBatchExecutor

deps_pip = ct.DepsPip(
  packages=["numpy==1.23.2", "scikit-learn==1.1.2"]
)

executor = AWSBatchExecutor(
    region='us-east-1',
    s3_bucket_name='<s3_bucket_name>',
    batch_queue='<batch_queue>',
    batch_job_role_name='<batch_job_role_name>',
    batch_job_log_group_name='<batch_job_log_group_name>',
    batch_execution_role_name='<batch_execution_role_name>',
    vcpu=2,
    memory=3.75,
    time_limit=300
)

To use any executor plugins we add the `executor` key to the `@electron` decorator, for example:

```python
@ct.electron(
    executor=executor
)
def train_svm(data, C, gamma):
    X, y = data
    clf = svm.SVC(C=C, gamma=gamma)
    clf.fit(X[90:], y[90:])
    return clf
```

In [ ]:
# Use executor plugin to train our SVM model.
from numpy.random import permutation
from sklearn import svm, datasets
import covalent as ct

@ct.electron(
    executor=executor,
    deps_pip=deps_pip
)
def train_svm(data, C, gamma):
    X, y = data
    clf = svm.SVC(C=C, gamma=gamma)
    clf.fit(X[90:], y[90:])
    return clf

@ct.electron
def load_data():
    iris = datasets.load_iris()
    perm = permutation(iris.target.size)
    iris.data = iris.data[perm]
    iris.target = iris.target[perm]
    return iris.data, iris.target

@ct.electron
def score_svm(data, clf):
    X_test, y_test = data
    return clf.score(
      X_test[:90],
    y_test[:90]
    )

@ct.lattice
def run_experiment(C=1.0, gamma=0.7):
    data = load_data()
    clf = train_svm(
      data=data,
      C=C,
      gamma=gamma
    )
    score = score_svm(
      data=data,
    clf=clf
    )
    return score

# Dispatch the workflow
dispatch_id = ct.dispatch(run_experiment)(
  C=1.0,
  gamma=0.7
)

# Wait for our result and get result value
result = ct.get_result(dispatch_id=dispatch_id, wait=True).result

print(result)

## Using Self Hosted Version of Covalent

In order to deploy the self-hosted instance of covalent we will require Terraform in order to programatically create AWS Resources such as VMs (EC2) instances, and other resources required for using the AWS Executors.

The installation guide for Terraform is the following: 
- https://developer.hashicorp.com/terraform/tutorials/aws-get-started/install-cli#install-terraform

We will now run through an example deployment of Covalent to AWS, the following repository contains all of the instructions for going about this:
- https://github.com/AgnostiqHQ/covalent-terraform

In [ ]:
import covalent as ct

ct.set_config("dispatcher.address", "<URL_FROM_TERRAFORM>")
ct.set_config("dispatcher.port", "48008")

### Important! Since now covalent is now running remotely we will need to install all dependencies required for each task

```python
@ct.electron(deps_pip=ct.DepsPip(packages=["numpy==1.23.2"]))
def my_task():
    return np.random.rand(3,2)
```

### Furthermore, executor plugins on remote covalent are pre-configured so they can be referred to simply as:

```python
@ct.electron(executor="awsbatch")
def my_batch_task():
    pass
```

The following are supported executors on hosted covalent:

- AWS Batch used as `"awsbatch"`
- AWS Lambda used as `"awslambda"`
- AWS Braket used as `"braket"`

#### To use Amazon Braket in a new account you must visit
- https://console.aws.amazon.com/braket/home?#/permissions?tab=executionRoles
To create a service-linked role

In [ ]:
# Import Executor (AWS Batch) - Note: we are using aliased executors

import covalent as ct
from covalent.executor import AWSBatchExecutor

deps_pip = ct.DepsPip(
  packages=["numpy==1.23.2", "scikit-learn==1.1.2"]
)

# As before we use the executor and install dependencies
@ct.electron(
    executor="awsbatch",
    deps_pip=deps_pip
)
def train_svm(data, C, gamma):
    X, y = data
    clf = svm.SVC(C=C, gamma=gamma)
    clf.fit(X[90:], y[90:])
    return clf

# IMPORTANT now since covalent is running remotely we must install the dependencies each task.
@ct.electron(
    deps_pip=deps_pip
)
def load_data():
    iris = datasets.load_iris()
    perm = permutation(iris.target.size)
    iris.data = iris.data[perm]
    iris.target = iris.target[perm]
    return iris.data, iris.target

@ct.electron(
    deps_pip=deps_pip
)
def score_svm(data, clf):
    X_test, y_test = data
    return clf.score(
      X_test[:90],
    y_test[:90]
    )

@ct.lattice
def run_experiment(C=1.0, gamma=0.7):
    data = load_data()
    clf = train_svm(
      data=data,
      C=C,
      gamma=gamma
    )
    score = score_svm(
      data=data,
    clf=clf
    )
    return score

# Dispatch the workflow
dispatch_id = ct.dispatch(run_experiment)(
  C=1.0,
  gamma=0.7
)

# Wait for our result and get result value
result = ct.get_result(dispatch_id=dispatch_id, wait=True).result

print(result)

### To revert to local covalent you can use the following

In [ ]:
import covalent as ct

ct.set_config("dispatcher.address", "localhost")
ct.set_config("dispatcher.port", "48008")


## Quiskit & Pennylane Quantum Circuit Example



Install qiskit & pennylane as well as the pennylane braket plugin:

In [ ]:
!pip install qiskit pennylane amazon-braket-pennylane-plugin

In [ ]:
"""
Compare expectation value estimates for 'M = I ⊗ Z' on the final state of the circuit,
     ┌────┐
q_0: ┤ Rz ├──■────────
     └────┘┌─┴─┐┌────┐
q_1: ──────┤ X ├┤ Ry ├
           └───┘└────┘
for arbitrary Z- and Y-rotation angles, as implemented in qiskit versus pennylane.
"""
from math import pi as PI
from typing import List, Tuple

import covalent as ct
import pennylane as qml
from qiskit import Aer, QuantumCircuit
from qiskit.opflow import CircuitStateFn, I, Z
from qiskit.opflow.state_fns import StateFn
from qiskit.opflow.expectations import PauliExpectation
from qiskit.opflow.converters import CircuitSampler

SHOTS = 10_000

# use the local braket backend for pennylane
QML_BRAKET_DEVICE = qml.device("braket.local.qubit", wires=2, shots=SHOTS)

# NOTE: use the Aer simulator instead of Quantinuum backend for now
QISKIT_BACKEND = Aer.get_backend("aer_simulator_statevector", shots=SHOTS)



@ct.electron(deps_pip=ct.DepsPip(packages=["qiskit"]))
def get_expval_from_qiskit(phi_z: float, phi_y: float) -> float:
    """qiskit implementation of the simulation"""
    qiskit_circuit = QuantumCircuit(2)
    qiskit_circuit.rz(phi_z, 0)
    qiskit_circuit.cnot(0, 1)
    qiskit_circuit.ry(phi_y, 1)

    psi = CircuitStateFn(qiskit_circuit)
    op = Z ^ I
    measureable_expr = StateFn(op, is_measurement=True).compose(psi)
    expectation = PauliExpectation().convert(measureable_expr)
    sampler = CircuitSampler(QISKIT_BACKEND).convert(expectation)
    return sampler.eval().real


@ct.electron(deps_pip=ct.DepsPip(packages=["pennylane"]))
def get_expval_from_pennylane(phi_z: float, phi_y: float) -> float:
    """pennylane implementation of the simulation"""
    @qml.qnode(device=QML_BRAKET_DEVICE)
    def _circuit():
        qml.RZ(phi_z, wires=0)
        qml.CNOT(wires=[0, 1])
        qml.RY(phi_y, wires=1)
        return qml.expval(qml.PauliZ(1))

    return float(_circuit())


@ct.electron
def mse(expvals_1: List[float], expvals_2: List[float]) -> float:
    """calculates MSE between two lists of floats"""
    assert len(expvals_1) == len(expvals_2)
    squared_error_sum = 0.0
    for e1, e2 in zip(expvals_1, expvals_2):
        squared_error_sum += (e1 - e2)**2
    return squared_error_sum / len(expvals_1)


@ct.lattice
def sweep_angles(angles: List[Tuple[float, float]]) -> float:
    """
    Get exp. values from qiskit and pennylane functions implementing the same
    quantum circuit and compare the outputs based on MSE

    Parameters
    ----------
    angles : List[Tuple[float, float]]
        pairs of angle parameters (phi_z, phi_y) for the quantum circuit

    Returns
    -------
    float
        mean squared error between qiskit and pennylane circuit results (should be ~0)
    """
    expvals_qiskit = []
    expvals_pennylane = []
    for phi_z, phi_y in angles:
        expvals_qiskit.append(get_expval_from_qiskit(phi_z, phi_y))
        expvals_pennylane.append(get_expval_from_pennylane(phi_z, phi_y))

    return mse(expvals_qiskit, expvals_pennylane)


if __name__ == "__main__":
    inputs = [(PI, PI / 2), (PI / 3, PI / 4), (PI / 5, PI / 6), (PI / 7, PI / 8)]
    dispatch_id = ct.dispatch(sweep_angles)(inputs)
    mean_squared_error = ct.get_result(dispatch_id, wait=True).result
    print(f"MSE: {mean_squared_error:.4f}")